In [5]:
import numpy as np
import pandas as pd
#import random
# import time
import matplotlib.pyplot as plt

plt.style.reload_library()

# from sktime.classification.interval_based import RandomIntervalSpectralEnsemble
# from sktime.classification.dictionary_based import ContractableBOSS, BOSSEnsemble
# from sktime.datatypes._panel._convert import from_2d_array_to_nested
import sklearn

from sklearn.model_selection import train_test_split
# from sklearn.ensemble import ExtraTreesClassifier
# from sklearn.preprocessing import StandardScaler, normalize
from sklearn.neighbors import KNeighborsClassifier
# from sklearn.pipeline import make_pipeline
# from sklearn.linear_model import RidgeClassifierCV
from sklearn.neighbors import KNeighborsClassifier
from wildboar.datasets import list_datasets, load_dataset
from wildboar.ensemble import ShapeletForestClassifier

# import pywt
# from pywt import wavedec, waverec

from tqdm.notebook import tqdm

# import pickle

# UCR time series datasets: https://www.cs.ucr.edu/%7Eeamonn/time_series_data_2018/

# UCR time series datasets: https://www.cs.ucr.edu/%7Eeamonn/time_series_data_2018/

In [6]:
knn_metrics = sklearn.neighbors.VALID_METRICS['brute']
print(knn_metrics)
knn_metrics = ['cityblock', 'cosine', 'euclidean', 'nan_euclidean']
knn_metric = 'euclidean'
print(knn_metric)

['cityblock', 'cosine', 'euclidean', 'haversine', 'l2', 'l1', 'manhattan', 'precomputed', 'nan_euclidean', 'braycurtis', 'canberra', 'chebyshev', 'correlation', 'cosine', 'dice', 'hamming', 'jaccard', 'kulsinski', 'mahalanobis', 'matching', 'minkowski', 'rogerstanimoto', 'russellrao', 'seuclidean', 'sokalmichener', 'sokalsneath', 'sqeuclidean', 'yule', 'wminkowski']
euclidean


In [7]:
clf_kNN = KNeighborsClassifier(metric=knn_metric)
print(clf_kNN)

KNeighborsClassifier(metric='euclidean')


In [8]:
UCR_datasets = list_datasets(repository='wildboar/ucr')

In [9]:
dataset_info = pd.DataFrame(columns=['size', 'classes', 'length'], index=UCR_datasets, dtype=float)

for dataset in tqdm(UCR_datasets):
    x_all, y_all = load_dataset(dataset, repository='wildboar/ucr')

    # remove rows wirandomth missing values
    x = x_all[~np.isnan(x_all).any(axis=1)]
    y = y_all[~np.isnan(x_all).any(axis=1)]

    classes = np.unique(y)  # all class labels
    total_examples, ts_length = x.shape

    dataset_info.loc[dataset] = [total_examples, len(classes), ts_length]

## Drop rows that have a value of zero in a column
dataset_info = dataset_info.loc[~(dataset_info == 0).any(axis=1)]
dataset_info = dataset_info.loc[~(dataset_info == 1).any(axis=1)]

dataset_info.to_pickle('data/datasets_information.pkl')

  0%|          | 0/128 [00:00<?, ?it/s]

In [10]:
dataset_info = pd.read_pickle('data/datasets_information.pkl')

In [11]:
dataset_info

,size,classes,length
ACSF1,200.0,10.0,1460.0
Adiac,781.0,37.0,176.0
ArrowHead,211.0,3.0,251.0
BME,180.0,3.0,128.0
Beef,60.0,5.0,470.0
...,...,...,...
Wine,111.0,2.0,234.0
WordSynonyms,905.0,25.0,270.0
Worms,258.0,5.0,900.0
WormsTwoClass,258.0,2.0,900.0


In [12]:
dataset_info.describe()

,size,classes,length
count,117.000000,117.000000,117.000000
mean,1581.572650,8.264957,537.102564
std,2996.395773,12.256872,583.137433
min,40.000000,2.000000,15.000000
25%,258.000000,2.000000,136.000000
50%,724.000000,3.000000,301.000000
75%,1272.000000,7.000000,720.000000
max,24000.000000,60.000000,2844.000000


In [13]:
dataset_info.quantile(.70)

size       1035.2
classes       6.2
length      571.4
Name: 0.7, dtype: float64

In [14]:
# # select subset of all datasets
# lb_size = 1
# up_size = 200
# lb_length = 10
# up_length = 1000
# lb_class = 2
# up_class = 8
#
# dataset_info = pd.read_pickle('data/datasets_information.pkl')
#
# selected = dataset_info.loc[
#     (dataset_info['size'] >= lb_size) & (dataset_info['size'] <= up_size) & (dataset_info['classes'] >= lb_class) & (
#             dataset_info['classes'] <= up_class) & (dataset_info['length'] >= lb_length) & (
#             dataset_info['length'] <= up_length)]
#
#
#
# # condition = (dataset_info['classes'] <= classes_ub) & (dataset_info['classes'] >= classes_lb)
# # selected = dataset_info.loc[condition]


In [91]:
dataset_info

selected_datasets = dataset_info.index.to_list()

index_dataset = 9

dataset = selected_datasets[index_dataset]
dataset

'Chinatown'

In [15]:
from wildboar.explain.counterfactual import counterfactuals

In [16]:
import plotly.express as px

# df = px.data.stocks()


In [93]:
selected_datasets = dataset_info.index.to_list()

index_dataset = 1

dataset = selected_datasets[index_dataset]

x_all, y_all = load_dataset(dataset, repository='wildboar/ucr')

# remove rows wirandomth missing values
x = x_all[~np.isnan(x_all).any(axis=1)]
y = y_all[~np.isnan(x_all).any(axis=1)]

classes = np.unique(y)  # all class labels
total_examples, ts_length = x.shape
print(classes)
print([{"label": i, "value": i} for i in classes])
x_ind = np.arange(total_examples)

x_train_ind, x_test_ind, y_train, y_test = train_test_split(x_ind, y, test_size=.30, random_state=0, shuffle=True,
                                                            stratify=None)

x_train = x[x_train_ind, :]
x_test = x[x_test_ind, :]
# print("x_train")
# print(x_train)
# print("x_test")
# print(x_test)
desired_class = 1.0
example_class = 2.0
temp_example_class = example_class
# y_train[y_train != example_class] = -1.0
# y_test[y_test != example_class] = -1.0
# print(len(y_train))
# print(len(y_test))
# print("len(x_train)")
# print(len(x_train))
# print("len(x_test)")
# print(len(x_test))
y_train[y_train == example_class] = -2.0
y_test[y_test == example_class] = -2.0
example_class = -2.0

y_train[(y_train == desired_class) & (y_train != example_class)] = -1.0
y_test[(y_test == desired_class) & (y_test != example_class)] = -1.0

x_train = np.delete(x_train, np.argwhere((y_train != -1.0) & (y_train != example_class)), axis=0)
x_test = np.delete(x_test, np.argwhere((y_test != -1.0) & (y_test != example_class)), axis=0)

y_train = np.delete(y_train, np.argwhere((y_train != -1.0) & (y_train != example_class)))
y_test = np.delete(y_test, np.argwhere((y_test != -1.0) & (y_test != example_class)))

y_train[y_train == example_class] = 1.0
y_test[y_test == example_class] = 1.0
# arr[(arr > 180) & (arr < 240) | (arr == 170)] = 1
# print(y_train)
# print(y_test)
# Delete values from x based on indexes from y and delete values from y that we do not want
# print(len(y_train))
# print(len(y_test))
# print("len(x_train)")
# print(len(x_train))
# print("len(x_test)")
# print(len(x_test))

# y_train = [x for x in xs if x != 'b']
# y_test = [x for x in xs if x != 'b']

clf_kNN = KNeighborsClassifier(metric="euclidean")
clf_kNN.fit(x_train, y_train)

#clf_RSF = ShapeletForestClassifier(n_estimators=50, metric='scaled_euclidean')
clf_RSF = ShapeletForestClassifier(
    n_estimators=20,
    metric='euclidean',
    max_depth=5,
    max_shapelet_size=.4,  # INTERACTION: Make this as input from user
    random_state=1,
)
clf_RSF.n_features_in_ = x_train.shape[-1]
clf_RSF.fit(x_train, y_train)

x_counterfactual_RSF, x_valid_RSF, x_score_RSF = counterfactuals(
    clf_RSF,
    x_test,
    -y_test,  # invert the classes, i.e., transform 1 -> -1 and -1 -> 1
    scoring="euclidean",
    valid_scoring=False,
    random_state=2,
    epsilon=1,
)
x_counterfactual_kNN, x_valid_kNN, x_score_kNN = counterfactuals(
    clf_kNN,
    x_test,
    -y_test,  # invert the classes, i.e., transform 1 -> -1 and -1 -> 1
    scoring="euclidean",
    valid_scoring=False,
)


[ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17. 18.
 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32. 33. 34. 35. 36.
 37.]
[{'label': 1.0, 'value': 1.0}, {'label': 2.0, 'value': 2.0}, {'label': 3.0, 'value': 3.0}, {'label': 4.0, 'value': 4.0}, {'label': 5.0, 'value': 5.0}, {'label': 6.0, 'value': 6.0}, {'label': 7.0, 'value': 7.0}, {'label': 8.0, 'value': 8.0}, {'label': 9.0, 'value': 9.0}, {'label': 10.0, 'value': 10.0}, {'label': 11.0, 'value': 11.0}, {'label': 12.0, 'value': 12.0}, {'label': 13.0, 'value': 13.0}, {'label': 14.0, 'value': 14.0}, {'label': 15.0, 'value': 15.0}, {'label': 16.0, 'value': 16.0}, {'label': 17.0, 'value': 17.0}, {'label': 18.0, 'value': 18.0}, {'label': 19.0, 'value': 19.0}, {'label': 20.0, 'value': 20.0}, {'label': 21.0, 'value': 21.0}, {'label': 22.0, 'value': 22.0}, {'label': 23.0, 'value': 23.0}, {'label': 24.0, 'value': 24.0}, {'label': 25.0, 'value': 25.0}, {'label': 26.0, 'value': 26.0}, {'label': 27.0, 'value': 27.0}, {

In [97]:
x_counterfactual_RSF.shape[0]
x_test.shape

(20, 176)

In [94]:
sel_instance_idx = 10
x_counter_RSF = x_counterfactual_RSF[sel_instance_idx, :]
x_counter_kNN = x_counterfactual_kNN[sel_instance_idx, :]
x_counter_test = x_test[sel_instance_idx, :]
y1 = x_counter_RSF
y2 = x_counter_kNN
y3 = x_counter_test

df = pd.DataFrame(dict(
    length=np.arange(y1.shape[0]),
    RSF_counter=y1,
    kNN_counter=y2,
    Data_Point=y3))

fig = px.line(df, x=df["length"], y=df.columns)
fig.show()

In [72]:
sel_instance_idx = 10
x_counter_RSF = x_counterfactual_RSF[sel_instance_idx, :]
x_counter_kNN = x_counterfactual_kNN[sel_instance_idx, :]
x_counter_test = x_test[sel_instance_idx, :]
y1 = x_counter_RSF
y2 = x_counter_kNN
y3 = x_counter_test


In [ ]:
x_counter_RSF

In [ ]:
df = pd.DataFrame(dict(
    length=np.arange(y1.shape[0]),
    x_counter_RSF=y1,
    x_counter_kNN=y2,
    x_test=y3
))

In [73]:
sel_instance_idx = 5
x_counter_RSF = x_counterfactual_RSF[sel_instance_idx, :]
x_counter_kNN = x_counterfactual_kNN[sel_instance_idx, :]
y1 = x_counterfactual_RSF[sel_instance_idx, :]
y2 = x_counterfactual_kNN[sel_instance_idx, :]
y3 = x_test[sel_instance_idx, :]
X = np.array([y1, y2, y3])
X = X.T
cols = ["x_counter_RSF", "x_counter_kNN", "x_test"]
df = pd.DataFrame(X, columns=cols)
df = df.T
display(df)
# df.columns = df.iloc[-1]
# print(df)
# df = df.head(-1)

# df.index.name = 'Country'

# Want time on the x-axis? ###
# just include:
# df = df.T
##############################

# plotly
fig = px.line(df, x=df.index, y=df.columns)
fig.update_layout(template="plotly_dark")
fig.show()

,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
x_counter_RSF,0.019997,0.013389,0.050173,0.031110,0.017921,0.050113,0.040883,0.047123,0.044335,0.007570,...,0.044135,0.038880,0.047283,0.026936,0.007765,0.012871,0.039896,0.010400,0.046678,0.006963
x_counter_kNN,0.030718,0.022954,0.033256,0.024429,0.026811,0.023753,0.087344,0.178649,0.358125,0.612625,...,0.027433,0.028144,0.017094,0.017391,0.023043,0.023368,0.022835,0.018562,0.025615,0.025141
x_test,0.019997,0.013389,0.050173,0.031110,0.017921,0.050113,0.040883,0.047123,0.044335,0.007570,...,0.044135,0.038880,0.047283,0.026936,0.007765,0.012871,0.039896,0.010400,0.046678,0.006963


/Users/arvinrastegar/opt/anaconda3/envs/Extremum/lib/python3.7/site-packages/plotly/express/_core.py:1222: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`



In [55]:
df

x_test,0.023094,0.039898,0.011821,0.002965,-0.000747,0.023448,0.009273,0.123000,0.455000,0.792000,...,0.023289,0.017181,0.003925,0.010830,0.025278,0.032108,0.036226,0.030195,0.017082,0.014414
Country,,,,,,,,,,,,,,,,,,,,,
x_counter_RSF,0.023094,0.039898,0.011821,0.002965,-0.000747,0.023448,0.009273,0.12300,0.455000,0.792000,...,0.376414,0.373038,0.364174,0.369213,0.378946,0.384471,0.389368,0.386318,0.377019,0.374531
x_counter_kNN,0.021178,0.023945,0.032206,0.020446,0.031415,0.012058,0.031515,0.01636,0.013733,0.032101,...,0.284500,0.132750,0.077906,0.056329,0.036121,0.036203,0.017755,0.025897,0.028528,0.016795


In [38]:
X = X.T

In [ ]:
X.shape

In [65]:
df

Year,2020,2021,2022,2023,2024,2025
Country,,,,,,
Canada,0.0,-0.252092,-1.161325,-1.491194,-1.850023,-2.550202
France,0.0,0.10961,-0.482027,-1.674791,-1.07132,0.080071
Germany,0.0,1.582481,1.770084,1.565208,-0.099581,1.75775
